In [27]:
import pandas as pd
from gensim import corpora, models

from collections import defaultdict
from pprint import pprint

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [2]:
data = pd.read_csv("data_cleaned/combined.csv")
data.head()

,id,title,format,affiliation 1,affiliation 2,affiliation 3,affiliation 4,affiliation 5,affiliation 6,affiliation 7,affiliation 8,affiliation 9,affiliation 10,tags,year,ID,Abstract,lower_abstract
0,2017-1,Assessing and Communicating Library Contributi...,preconference,University of Illinois at Urbana-Champaign,Loyola Marymount University,Anne Arundel Community College,Dominican University,NaN,NaN,NaN,NaN,NaN,NaN,Assessment,2017,2017-1,Higher education institutions of all types are...,higher education institutions of all types are...
1,2017-2,COUNTER Bootcamp: A Workshop about COUNTER Rep...,preconference,Virtual Library of Virginia,University of Richmond,EBSCO Information Services,Mongomery College,NaN,NaN,NaN,NaN,NaN,NaN,Assessment,2017,2017-2,Take a deep dive into the process and workflow...,take a deep dive into the process and workflow...
2,2017-3,Information Literacy Instruction Transformed: ...,preconference,Towson University,University of North Carolina Greensboro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Teaching and Learning,2017,2017-3,Universal Design for Learning (UDL) offers ins...,universal design for learning (udl) offers ins...
3,2017-4,Law School for Librarians: A Tangled Web of Co...,preconference,Purdue University,University of Louisville,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Scholarly Communication,2017,2017-4,Librarians encounter legal and policy issues n...,librarians encounter legal and policy issues n...
4,2017-5,"Make It, Map It, Take It: Create Your Own Digi...",preconference,California State University Dominguez Hills,University of California Irvine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Teaching and Learning,2017,2017-5,"Go beyond Camtasia, bring your laptop, and lea...","go beyond camtasia, bring your laptop, and lea..."


In [3]:
documents = data["lower_abstract"].tolist()

In [5]:
stoplist = set('for a of the and to in'.split())

In [6]:
texts = [[word for word in document.lower().split() if word not in stoplist]
            for document in documents]

In [9]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

In [10]:
texts = [[token for token in text if frequency[token] > 1]
            for text in texts]

In [13]:
pprint(texts[:1])

[['higher',
  'education',
  'institutions',
  'all',
  'types',
  'are',
  'facing',
  'intensified',
  'attention',
  'assessment',
  'accountability',
  'issues.',
  'academic',
  'libraries',
  'are',
  'increasingly',
  'connecting',
  'with',
  'colleagues',
  'campus',
  'stakeholders',
  'design',
  'implement',
  'assessment',
  'that',
  'documents',
  'their',
  'contributions',
  'institutional',
  'priorities.',
  'this',
  'day-long',
  'workshop',
  'on',
  'strategic',
  'sustainable',
  'assessment,',
  'participants',
  'will',
  'identify',
  'institutional',
  'priorities',
  'campus',
  'partners,',
  'design',
  'an',
  'assessment',
  'project',
  'grounded',
  'action',
  'research,',
  'prepare',
  'plan',
  'communicating',
  'project',
  'results.',
  'this',
  'event',
  'is',
  'based',
  'on',
  'highly',
  'successful',
  'acrl',
  'assessment',
  'action',
  'program',
  'curriculum.']]


In [15]:
dictionary = corpora.Dictionary(texts)
dictionary.save('acrl.dict')

In [17]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('acrl.mm', corpus)

In [20]:
hdp = models.HdpModel(corpus, id2word=dictionary)

In [24]:
hdp.print_topics()

[(0,
  '0.004*necessary + 0.003*transition + 0.003*on + 0.003*students. + 0.003*states + 0.003*does + 0.003*topics, + 0.003*brief + 0.003*this + 0.003*themes'),
 (1,
  '0.004*this + 0.004*long-term + 0.004*insight + 0.003*inventory + 0.003*decision + 0.003*pedagogy. + 0.003*intensified + 0.003*assessment + 0.003*uniquely + 0.003*composting'),
 (2,
  '0.004*partners + 0.004*taken + 0.003*there + 0.003*problem-based + 0.003*archives + 0.003*supporting + 0.003*writing, + 0.003*innovations + 0.003*pilot + 0.003*liaisons'),
 (3,
  '0.004*valuable + 0.004*changing + 0.004*threshold + 0.003*site + 0.003*recently + 0.003*women + 0.003*uncover + 0.003*emphasizes + 0.003*surprising + 0.003*methods,'),
 (4,
  '0.006*kingdom. + 0.004*paid + 0.004*reduce + 0.004*administrative + 0.003*structural + 0.003*minutes + 0.003*built-in + 0.003*technologies + 0.003*environmental + 0.003*as'),
 (5,
  '0.004*engage + 0.004*score + 0.004*publisher, + 0.003*shift + 0.003*power + 0.003*percent + 0.003*oer + 0.00

In [28]:
pyLDAvis.enable_notebook()

In [29]:
gensimvis.prepare(hdp, corpus, dictionary)

/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentat

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12     0.024351  0.029743       1        1  10.202423
0     -0.000158  0.016300       2        1   6.770349
1      0.006146 -0.013840       3        1   6.535865
33    -0.007812  0.003231       4        1   4.668852
39    -0.013053 -0.006240       5        1   3.773655
...         ...       ...     ...      ...        ...
132    0.008754  0.029629     146        1   0.001932
142    0.003284 -0.006445     147        1   0.001532
145   -0.007046  0.013439     148        1   0.000802
143   -0.031077 -0.005621     149        1   0.000753
147    0.028362 -0.021697     150        1   0.000667

[150 rows x 5 columns], topic_info=             Term       Freq      Total  Category  logprob  loglift
52           this  19.000000  19.000000   Default  30.0000  30.0000
1155     measured  10.000000  10.000000   Default  29.0000  29.0000
124         tools  12.000000  12.000000   Default  28.0000  28.0000
976     promoting  10.000000  10.000000   Default  27.0000  27.0000
1604       joseph  10.000000  10.000000   Default  26.0000  26.0000
...           ...        ...        ...       ...      ...      ...
880   effectively   0.000203   7.326679  Topic150  -6.0452   1.4221
1495    inventory   0.000224  10.340756  Topic150  -5.9450   1.1778
1060   approaches   0.000200   7.484943  Topic150  -6.0562   1.3897
129       broader   0.000201   7.966431  Topic150  -6.0520   1.3316
594       address   0.000203   8.851277  Topic150  -6.0406   1.2377

[5804 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1621      1  0.113904        &
1621      5  0.113904        &
1621      6  0.113904        &
1621      7  0.113904        &
1621     14  0.113904        &
...     ...       ...      ...
628       4  0.163025  youtube
629       2  0.144894   you’ll
629       9  0.144894   you’ll
1585      2  0.314278   you’re
1585     14  0.157139   you’re

[3846 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 1, 2, 34, 40, 10, 41, 48, 14, 91, 11, 68, 4, 97, 8, 16, 9, 73, 27, 69, 20, 7, 57, 61, 122, 3, 142, 117, 118, 31, 60, 51, 81, 149, 37, 55, 136, 77, 58, 36, 105, 5, 124, 12, 39, 21, 104, 65, 76, 106, 50, 95, 46, 47, 59, 103, 66, 101, 19, 18, 43, 102, 86, 115, 17, 24, 88, 70, 23, 99, 15, 64, 25, 121, 138, 62, 128, 100, 78, 38, 49, 67, 82, 114, 29, 98, 6, 71, 90, 74, 63, 147, 135, 84, 140, 89, 112, 119, 109, 110, 28, 116, 94, 150, 22, 126, 145, 45, 26, 125, 130, 44, 52, 87, 137, 134, 79, 139, 80, 123, 96, 42, 30, 92, 75, 32, 72, 33, 35, 53, 56, 54, 85, 93, 83, 111, 108, 107, 131, 127, 113, 132, 120, 141, 129, 133, 143, 146, 144, 148])

In [30]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=30)

In [31]:
lda.print_topics()

[(27,
  '0.018*"will" + 0.018*"we" + 0.016*"from" + 0.014*"sabotage" + 0.014*"behaviors" + 0.014*"organizational" + 0.014*"field" + 0.012*"this" + 0.012*"new" + 0.011*"by"'),
 (16,
  '0.015*"on" + 0.014*"is" + 0.014*"will" + 0.013*"library" + 0.012*"this" + 0.011*"with" + 0.011*"libraries" + 0.011*"open" + 0.009*"learning" + 0.009*"that"'),
 (21,
  '0.040*"no" + 0.040*"abstract" + 0.018*"how" + 0.015*"faculty" + 0.014*"are" + 0.012*"literacy" + 0.012*"learn" + 0.011*"program" + 0.011*"information" + 0.010*"this"'),
 (1,
  '0.022*"as" + 0.017*"will" + 0.017*"libraries" + 0.016*"diversity" + 0.016*"are" + 0.015*"this" + 0.014*"with" + 0.014*"how" + 0.010*"an" + 0.010*"about"'),
 (7,
  '0.023*"their" + 0.022*"will" + 0.015*"participants" + 0.012*"from" + 0.012*"how" + 0.012*"information" + 0.011*"environments." + 0.011*"learning" + 0.010*"design" + 0.010*"on"'),
 (26,
  '0.069*"abstract" + 0.069*"no" + 0.015*"information" + 0.014*"as" + 0.012*"literacy" + 0.011*"an" + 0.010*"libraries" + 

In [32]:
gensimvis.prepare(lda, corpus, dictionary)

/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/scott/projects/acrl_analysis/acrl/acrl_env/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentat

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0     -0.020623  0.035222       1        1  9.519855
25    -0.078140  0.013897       2        1  7.808661
22    -0.040873  0.053794       3        1  7.569671
29    -0.056300  0.042906       4        1  6.495129
23    -0.050298  0.038018       5        1  6.374626
15    -0.061946  0.044787       6        1  5.323527
1     -0.048594 -0.006061       7        1  4.829143
3     -0.053102  0.016786       8        1  4.477844
8     -0.025409  0.030903       9        1  4.380035
9      0.090511  0.021955      10        1  4.312564
16    -0.073970  0.037103      11        1  3.584515
2     -0.029249  0.032535      12        1  3.306875
24    -0.075470  0.020956      13        1  2.997224
10     0.173079 -0.041273      14        1  2.598606
12    -0.026661  0.046152      15        1  2.545957
5     -0.071982  0.014836      16        1  2.367325
21     0.098294  0.040876      17        1  2.159202
17     0.061719  0.059133      18        1  2.067620
26     0.098032  0.038722      19        1  1.963589
28    -0.030434 -0.050326      20        1  1.856346
18     0.106534  0.056469      21        1  1.753729
14     0.093210 -0.019837      22        1  1.707644
20    -0.051396 -0.068290      23        1  1.695988
13     0.030785 -0.034763      24        1  1.572760
11     0.034943 -0.042129      25        1  1.533361
19    -0.048771 -0.104569      26        1  1.512844
6      0.070263 -0.011268      27        1  1.306724
4     -0.037280 -0.115708      28        1  1.127600
7      0.015057 -0.043844      29        1  0.626627
27     0.008072 -0.106984      30        1  0.624407, topic_info=            Term        Freq       Total Category  logprob  loglift
704     abstract  172.000000  172.000000  Default  30.0000  30.0000
153           no  180.000000  180.000000  Default  29.0000  29.0000
54          will  221.000000  221.000000  Default  28.0000  28.0000
100  information   88.000000   88.000000  Default  27.0000  27.0000
108     literacy   55.000000   55.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
50          that    0.797909  128.359190  Topic30  -4.6085  -0.0045
58          also    0.582521   28.052045  Topic30  -4.9231   1.2017
176          how    0.702390  160.313490  Topic30  -4.7360  -0.3543
564          but    0.575028   28.009432  Topic30  -4.9361   1.1902
291           or    0.583509   33.520392  Topic30  -4.9214   1.0253

[1901 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1621     11  0.774787       &
1356      1  0.286435     (1)
1356      4  0.286435     (1)
1356     24  0.286435     (1)
1357      1  0.287668     (2)
...     ...       ...     ...
629       6  0.161582  you’ll
629       9  0.161582  you’ll
629      22  0.161582  you’ll
1585      2  0.345875  you’re
1585      9  0.345875  you’re

[4344 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 26, 23, 30, 24, 16, 2, 4, 9, 10, 17, 3, 25, 11, 13, 6, 22, 18, 27, 29, 19, 15, 21, 14, 12, 20, 7, 5, 8, 28])